In [ ]:
#!pip install fredapi

In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import time

# Prepare Fred object
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

fred_key = os.getenv('FRED_KEY')

from fredapi import Fred
fred = Fred(api_key=fred_key)

# Inflation and interest rate

### 1. Get the data

In [ ]:
# Data sets were chosen after browsing through different available Fred datasets
interest_rate = fred.get_series(series_id='REAINTRATREARAT10Y') # 10-Year Real Interest Rate, Monthly
inflation = fred.get_series(series_id='CORESTICKM159SFRBATL') # Sticky Price Consumer Price Index less Food and Energy, Monthly

### 2. Look at the data and format

In [160]:
inflation.shape

(687,)

In [162]:
interest_rate.shape

(519,)

In [163]:
inflation.head()

1967-12-01         NaN
1968-01-01    3.651861
1968-02-01    3.673819
1968-03-01    4.142164
1968-04-01    4.155828
dtype: float64

In [165]:
interest_rate.head()

1982-01-01    7.623742
1982-02-01    7.656648
1982-03-01    7.128993
1982-04-01    7.408347
1982-05-01    7.320041
dtype: float64

In [166]:
# Since we want to plot data on a same chart, let's look at the same timeframe for both data sets.
inflation = inflation.truncate(before=pd.Timestamp('1982-01-01'))

### 3. Import bokeh modules

In [21]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, CustomJS, Select, RadioButtonGroup, Div, HoverTool

### 4. Plot the data

In [167]:
# Plot inflation and interest rate
# Create a figure
p = figure(title="Inflation and Interest Rate",
            x_axis_label='Time',
            y_axis_label='Rate', 
            x_axis_type='datetime',
            width = 800, 
            height = 400)

p.line(interest_rate.index, interest_rate.values, legend_label="interest rate", line_width=1.5, color="navy")
p.line(inflation.index, inflation.values, legend_label="inflation", line_width=2, color="green")
p.legend.location = "top_right"
hover_tool = HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'),
        ( 'Rate',  '@y{int}' ),
    ],
    formatters={
        '@x': 'datetime', # use 'datetime' formatter for '@x' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    #mode = 'vline'
)
p.add_tools(hover_tool)
show(p)

# Unemployment rate by state

### 1. Get and format the data

In [20]:
# Search for unemployment per state datasets
unemp_df = fred.search('unemployment rate state', filter=('frequency','Monthly'))
unemp_df = unemp_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
unemp_df = unemp_df.loc[unemp_df['title'].str.contains('Unemployment Rate')]

all_results = []

for myid in unemp_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
    time.sleep(0.1) # Don't request to fast and get blocked
uemp_results = pd.concat(all_results, axis=1)


cols_to_drop = []
for i in uemp_results:
    if len(i) > 4:
        cols_to_drop.append(i)
uemp_results = uemp_results.drop(columns = cols_to_drop, axis=1)

uemp_states = uemp_results.copy()  #.drop('UNRATE', axis=1)
uemp_states = uemp_states.dropna()
id_to_state = unemp_df['title'].str.replace('Unemployment Rate in ','').to_dict()
uemp_states.columns = [id_to_state[c] for c in uemp_states.columns]
uemp_states['date']=uemp_states.index

### 2. Import additional bokeh modules

In [22]:
from bokeh.models import ColumnDataSource, Dropdown
from bokeh.io import output_notebook
from bokeh.models.callbacks import CustomJS

### 3. Plot the data

In [42]:
output_notebook()
# create the dropdown menu 
# (Note: Bokeh call's this select, because of the html nomenclature; there is also a difference Bokeh Dropdown)
dropdown = Select(title = '', 
    value = 'California',
    options = uemp_states.columns.unique().tolist(),
    width = 200,
    margin = (5, 5, 5, 80)
)

# define the tools you want to use
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

# define the tooltip
hover_tool = HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'),
        ( 'Rate',  '@y{int}' ),
    ],
    formatters={
        '@x': 'datetime', # use 'datetime' formatter for '@x' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    #mode = 'vline'
)

# create a new plot  
f = figure(tools = TOOLS, 
    width = 800, 
    height = 400, 
    x_range = [np.nanmin(uemp_states['date']), np.nanmax(uemp_states['date'])],
    y_range = [0, 30],
    x_axis_label = 'Date',
    y_axis_label = 'Unemployment rate'
)
f.tools.append(hover_tool)

source = ColumnDataSource(
    data = dict(
        x = uemp_states['date'], 
        y = uemp_states['California'], # included for the tooltips
    )
)

# fill the area
f.varea(x = 'x', y1 = 'y', y2 = 0,   
    source = source, 
    color = 'mediumblue', 
    alpha = 0.5
)

# draw the line
f.line('x', 'y',
    source = source, 
    color = 'black', 
    alpha = 1, 
    line_width = 2
)

allSource = ColumnDataSource(uemp_states)

# Javascript code for the callback
callback = CustomJS(
    args = dict(
        source = source, 
        allSource = allSource,
        dropdown = dropdown,
        figure = f,
        ranges = dict(
            x = f.x_range, 
            y = f.y_range
        ) 
    ),
    code = 
    """
        // get the value from the dropdown
        var state = dropdown.value;    
        
        // filter the full data set to include only selected state
        var x = allSource.data['date'];
        var y = allSource.data[state];

            //update the data in the plot
            source.data.x = x;
            source.data.y = y;
            source.change.emit();
            figure.title.text = 'Unemployment in ' + state;

            //reset the axis limits
            //note that this ... syntax may not work on all browsers
            ranges.x.start = Math.min(...x); 
            ranges.x.end =  Math.max(...x);
            ranges.y.start = Math.max(Math.min(...y),0); 
            ranges.y.end =  Math.max(...y) + 2;
        
    """
)

#attach the callback
f.title.text = 'Unemployment in ' + dropdown.value
dropdown.js_on_change('value', callback)

show(row([
    dropdown,
    f]
))

Loading BokehJS ...

# Unemployment rate by sex

### 1. Get and format the data

In [26]:
unemployment_men = fred.get_series(series_id='LNS14000001') # Unemployment Rate - Men, Monthly
unemployment_women = fred.get_series(series_id='LNS14000002') # Unemployment Rate - Women, Monthly

In [27]:
unemployment_men.shape

(926,)

In [28]:
unemployment_women.shape

(926,)

In [33]:
unemployment_women.head()

1948-01-01    3.3
1948-02-01    4.5
1948-03-01    4.4
1948-04-01    4.3
1948-05-01    3.7
dtype: float64

In [34]:
# Let's limit data to just last 5 years
unemployment_women = unemployment_women.truncate(before=pd.Timestamp('2019-01-01'))
unemployment_men = unemployment_men.truncate(before=pd.Timestamp('2019-01-01'))

### 2. Import additional bokeh modules

In [29]:
from bokeh.transform import factor_cmap

### 3. Plot the data

In [41]:
# Sample data
categories = unemployment_men.index

# Stacking data
data = {
    'dates': unemployment_men.index,
    'men': unemployment_men.values,
    'women': unemployment_women.values
}
source = ColumnDataSource(data)

# Define figure
p = figure(x_axis_type='datetime', title="Unemployment rate by sex",
           toolbar_location=None, tools="")

# Stack bars
colors = ["dodgerblue", "plum"]
p.vbar_stack(['men', 'women'], x='dates', width=86400000 * 7, color=colors,
             source=source, legend_label=['Men', 'Women'])

# Styling
p.y_range.start = 0
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "horizontal"
show(p)